In [22]:
from ipywidgets import IntProgress, VBox, HTML
from IPython.display import display

def display_loading_animation(text):
    progress = IntProgress(value=0, min=0, max=100, bar_style='',
    layout={'width': '300px', 'height': '11px'})
    progress_text = HTML(value="1%")

    display(VBox([progress, progress_text]))

    for i in range(101):
        progress.value = i
        progress_text.value = f"<b>{i}%</b>"
        for _ in range(10**5):
            pass

    progress.bar_style = 'success'
    progress_text.value = "<b>Done!</b>"
    print(text)



In [23]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
import random
from ipywidgets import Button, Output, VBox
from IPython.display import display, clear_output
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import Button, Output, VBox, Layout

class Adaline:
    def __init__(self, learning_rate=0.00001, n_iterations=485):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.errors_ = []

    def fit(self, X, y):
        self.weights = np.random.normal(0, 0.00001, size=(1 + X.shape[1]))

        for epoch in range(self.n_iterations):

            net_input = self.net_input(X)
            output = self.activation(net_input)
            errors = y - output

            self.weights[1:] += self.learning_rate * X.T.dot(errors)
            self.weights[0] += self.learning_rate * errors.sum()

            mse = np.mean(errors**2)
            self.errors_.append(mse)

            if mse < self.learning_rate:
                display_loading_animation(f"Early stopping at epoch  {epoch + 1}")
                break;
            # display_loading_animation(f"Epoch {epoch + 1}: Mean Error = {mse:.5f}, Weights Mean = {np.mean(self.weights):.5f}\n")
        return self

    def net_input(self, X):
        return np.dot(X, self.weights[1:]) + self.weights[0]

    def activation(self, X):
        return X

    def predict(self, X):
        return np.where(self.activation(self.net_input(X)) >= 0.5, 1, 0)


class Madaline:
    def __init__(self, n_adalines=3, learning_rate=0.00001, n_iterations=485):
        self.n_adalines = n_adalines
        self.adalines = [Adaline(learning_rate, n_iterations) for _ in range(n_adalines)]

    def fit(self, X, y):
        feature_split = np.array_split(X, self.n_adalines, axis=1)
        for i, adaline in enumerate(self.adalines):
            adaline.fit(feature_split[i], y)
        return self

    def predict(self, X):
        feature_split = np.array_split(X, self.n_adalines, axis=1)
        predictions = np.array([adaline.predict(split) for adaline, split in zip(self.adalines, feature_split)])
        return np.where(predictions.mean(axis=0) >= 0.5, 1, 0)


def generate_shape(shape_type, image_size=64, noise_level=0.1):
    image = np.zeros((image_size, image_size), dtype=np.uint8)
    center = (image_size // 2, image_size // 2)
    size = int(image_size * 0.3)

    if shape_type == 'circle':
        cv2.circle(image, center, size, 255, -1)
    elif shape_type == 'square':
        top_left = (center[0] - size, center[1] - size)
        bottom_right = (center[0] + size, center[1] + size)
        cv2.rectangle(image, top_left, bottom_right, 255, -1)
    elif shape_type == 'triangle':
        points = np.array([
            [center[0], center[1] - size],
            [center[0] - size, center[1] + size],
            [center[0] + size, center[1] + size]
        ], np.int32)
        cv2.fillPoly(image, [points], 255)
    elif shape_type == 'pentagon':
        points = np.array([
            [center[0], center[1] - size],
            [center[0] - size, center[1] - size // 2],
            [center[0] - size // 2, center[1] + size],
            [center[0] + size // 2, center[1] + size],
            [center[0] + size, center[1] - size // 2]
        ], np.int32)
        cv2.fillPoly(image, [points], 255)
    elif shape_type == 'hexagon':
        points = np.array([
            [center[0] - size, center[1]],
            [center[0] - size // 2, center[1] - size],
            [center[0] + size // 2, center[1] - size],
            [center[0] + size, center[1]],
            [center[0] + size // 2, center[1] + size],
            [center[0] - size // 2, center[1] + size]
        ], np.int32)
        cv2.fillPoly(image, [points], 255)

    noise = np.random.normal(0, noise_level, image.shape)
    noisy_image = image + noise * 255
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

def create_dataset(n_samples=485):
    X = []
    y = []
    shapes = ['circle', 'square', 'triangle', 'pentagon', 'hexagon']
    for _ in range(n_samples):
        shape = random.choice(shapes)
        image = generate_shape(shape)
        X.append(image.flatten())
        y.append([1 if shape == s else 0 for s in shapes])
    return np.array(X), np.array(y)


import pandas as pd

def save_dataset_to_csv(n_samples=485, filename="shapes_dataset.csv"):
    X, y = create_dataset(n_samples)

    shapes = ['circle', 'square', 'triangle', 'pentagon', 'hexagon']
    image_columns = [f'pixel_{i}' for i in range(X.shape[1])]
    label_columns = [f'label_{shape}' for shape in shapes]

    data = pd.DataFrame(X, columns=image_columns)
    labels = pd.DataFrame(y, columns=label_columns)
    dataset = pd.concat([data, labels], axis=1)

    dataset.to_csv(filename, index=False)
    print(f"Dataset saved to {filename}")

save_dataset_to_csv(n_samples=485, filename="shapes_dataset.csv")

def predict_shape(image, models):
    image_scaled = scaler.transform([image.flatten()])
    predictions = {}
    for shape, model in models.items():
        adaline_pred = model['adaline'].predict(image_scaled)[0]
        madaline_pred = model['madaline'].predict(image_scaled)[0]
        predictions[shape] = (adaline_pred + madaline_pred) / 2
    return max(predictions.items(), key=lambda x: x[1])[0]

def calculate_accuracy():
    correct = 0
    total = len(X_test)
    for i in range(total):
        actual_shape = shapes[np.argmax(y_test[i])]
        predicted_shape = predict_shape(X_test[i].reshape(64, 64), shape_models)
        if actual_shape == predicted_shape:
            correct += 1
    return correct / total


display_loading_animation("Initializing Shape Recognition System...")
X, y = create_dataset(485)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

shapes = ['circle', 'square', 'triangle', 'pentagon', 'hexagon']
shape_models = {}

for i, shape in enumerate(shapes):
    display_loading_animation(f"Training models for {shape}...")
    adaline = Adaline(learning_rate=0.00001, n_iterations=485)
    adaline.fit(X_train_scaled, y_train[:, i])
    madaline = Madaline(n_adalines=3, learning_rate=0.00001, n_iterations=485)
    madaline.fit(X_train_scaled, y_train[:, i])
    shape_models[shape] = {'adaline': adaline, 'madaline': madaline}

accuracy = calculate_accuracy()
display_loading_animation(f"\nModel Accuracy: {accuracy:.2%}")


output = Output()

def generate_shape_and_test(_):
    with output:
        clear_output(wait=True)
        random_shape = random.choice(shapes)
        test_image = generate_shape(random_shape)
        predicted_shape = predict_shape(test_image, shape_models)
        plt.imshow(test_image, cmap='plasma')
        plt.title(
    f"Generated: {random_shape}\nPredicted: {predicted_shape}",
    fontdict={
        'fontsize': 16,
        'fontweight': 'bold',
        'color': 'blue',
        'family': 'serif'
    },
    loc='center')
        plt.axis('off')
        plt.show()

def view_accuracy_and_plot(_):
    with output:
        clear_output(wait=True)
        print(f"\nCurrent Model Accuracy: {accuracy:.2%}")
        print("\nPlotting Error Progression for Each Shape...")

        plt.figure(figsize=(15, 10))
        for i, shape in enumerate(shapes):
            adaline_errors = shape_models[shape]['adaline'].errors_
            madaline_errors = []
            for adaline in shape_models[shape]['madaline'].adalines:
                madaline_errors.append(adaline.errors_)

            plt.subplot(len(shapes), 2, i * 2 + 1)
            plt.plot(adaline_errors, label=f'Adaline - {shape.capitalize()}')
            plt.title(f'{shape.capitalize()} - Adaline Error')
            plt.xlabel('Epochs')
            plt.ylabel('Mean Squared Error')
            plt.legend()

            plt.subplot(len(shapes), 2, i * 2 + 2)
            for j, errors in enumerate(madaline_errors):
                plt.plot(errors, label=f'Madaline Adaline-{j + 1}')
            plt.title(f'{shape.capitalize()} - Madaline Error')
            plt.xlabel('Epochs')
            plt.ylabel('Mean Squared Error')
            plt.legend()

        plt.tight_layout()
        plt.show()


def exit_program(_):
    with output:
        clear_output(wait=True)
        print("\033[94m" + "Thank you for using the Shape Recognition System!" + "\033[0m")
        print("\033[94m" + "Created by BRAHIM OUHAMMOU ,BENCHELHA AYOUB, HAJAR CHARKAOUI \nGoodbye!" + "\033[0m")

button1 = Button(description="Generate and Test Random Shape", layout=Layout(width='300px'), button_style='warning')

button2 = Button(description="View Model Accuracy and Errors", layout=Layout(width='300px'), button_style='info')

button3 = Button(description="Exit", layout=Layout(width='300px'), button_style='danger')

button1.on_click(generate_shape_and_test)
button2.on_click(view_accuracy_and_plot)
button3.on_click(exit_program)

display(VBox([button1, button2, button3]), output)


Dataset saved to shapes_dataset.csv


Initializing Shape Recognition System...


Training models for circle...


Training models for square...


Training models for triangle...


Training models for pentagon...


Training models for hexagon...



Model Accuracy: 100.00%


Output()